# Analysis of neuropixels data

Install libraries needed:

In [ ]:
!pip install rastermap 
!pip install mat73
# for plotting
!pip install matplotlib
!pip install opencv-python-headless

You will also need to install pytorch, see instructions here: https://pytorch.org/get-started/locally/

If you have a GPU available you will want to install the CUDA version of pytorch.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch 

# these files are in the same folder as the notebook
import prediction, datasets, process, summary

from process import mice, days 
mouse = mice[0]
day = days[0][0]

# path to data
root = '/media/carsen/ssd2/syncope/'

for i, mouse in enumerate(mice):
    for day in days[i]:
        process.process_dataset(root, mouse, day, device=torch.device('cuda'))

## neural activity analysis

compute summary statistics of latencies, firing rates etc

In [ ]:
process.get_stats(root)

visualize stats

In [ ]:
dat = np.load('summary_stats.npz', allow_pickle=True)
arrays = []
for f in dat.files:
    arrays.append(dat[f])

(latenciesS, rastersS, latenciesO, rastersO, latenciesC, rastersC, latenciesL, rastersL,
    suppresseds, laser_frs, areas, firing_rates, firing_variances, rasters_summary, timeoffs) = arrays

In [ ]:
area_names, sareas = np.unique(areas.copy(), return_inverse=True) 

imp.reload(datasets)
region_names, region, superregion_names, superregion = datasets.region_categories()

regions = np.zeros(len(sareas), 'int')
super_regions = np.zeros(len(sareas), 'int')

for s,an in enumerate(area_names):
    reg = np.array([an in r for r in region])
    ind = np.nonzero(reg)[0][0]
    regions[sareas==s] = ind
    rname = region_names[ind]
    sreg = np.array([rname in sr for sr in superregion])
    ind = np.nonzero(sreg)[0][0]
    super_regions[sareas==s] = ind
    

In [ ]:
sa = regions.copy()
n_reg = len(region_names)
fracS = np.nan * np.ones(n_reg)
fracC = np.nan * np.ones(n_reg)
fracL = np.nan * np.ones(n_reg)
latL = np.nan * np.ones(n_reg)
latS = np.nan * np.ones(n_reg)
toff = np.nan * np.ones(n_reg)
supp = np.nan * np.ones(n_reg)
bad_areas = ['VS', 'Fiber', 'error']
print('frac active \t syncope \t control')
for i in range(n_reg):
    if (sa==i).sum() > 0 and region_names[i] not in bad_areas:
        inds = np.nonzero(sa==i)[0]
        supp[i] = suppresseds[inds].mean()
        fracS[i] = (latenciesS[inds]<0.25).mean()
        fracL[i] = (latenciesL[inds]<0.25).mean()
        fracC[i] = (latenciesC[inds]<0.25).mean()
        latL[i] = np.median(latenciesL[inds][latenciesL[inds]<0.25])
        latS[i] = np.mean(latenciesS[inds][latenciesS[inds]<0.25])
        inds = inds[latenciesS[inds]<0.25]
        toff[i] = np.nanmean(timeoffs[inds])
        print(f'{region_names[i]}       \t {1-fracS[i]:.3f} \t\t {1-fracC[i]:.3f} ')    

plot rasters

In [ ]:
nr = len(superregion_names) - 2
lS = np.zeros(nr) * np.nan

for sr in np.arange(0, nr):
    ir = np.unique(regions[super_regions==sr])
    lS[sr] = np.nanmean(fracS[ir]) 
isort = lS.argsort()
isort= np.array([5, 1, 4, 6, 3, 0, 2])
isort_all = np.zeros(0, 'int')

colors = np.zeros((len(fracS), 4))
cmaps = ['summer', 'Blues', 'Oranges', 'winter', 'Reds', 'Purples', [1,0,1,1]]
istart = .3 * np.ones(len(cmaps))
iend = .9 * np.ones(len(cmaps))
istart[3] = 0.7
iend[3] = 1.0
iend[0] = 0.8
ylabels = []
for ks, sr in enumerate(isort):
    ir = np.unique(regions[super_regions==sr])
    ir = ir[fracS[ir].argsort()]
    ir = ir[~np.isnan(fracS[ir])]
    isort_all = np.append(isort_all, ir)
    if ks < len(isort)-1:
        colors[ir] = plt.get_cmap(cmaps[ks])(np.linspace(istart[ks], iend[ks], len(ir)))
    else:
        colors[ir[0]] = cmaps[ks]

neural_sort = np.zeros(0, 'int')
nk=0
for k,i in enumerate(isort_all):
    lata = latenciesS[sa==i].copy()
    raster = rastersS[sa==i].astype('float32')
    ylabels.append(region_names[i])
    rmax = raster.max(axis=1)
    rmax[rmax==0] = 1
    lata[lata >= 0.25] = np.nan
    
    inds = [~np.isnan(lata), np.isnan(lata)]
    for a in range(2):
        raster = rastersS[sa==i].astype('float32')[inds[a]]
        isort_raster = raster[:,:].mean(axis=1).argsort() #np.arange(0, raster.shape[0])#
        neural_sort = np.append(neural_sort, inds[a].nonzero()[0][isort_raster])
    nk+=raster.shape[0]
    

fig=plt.figure(figsize=(8,14), dpi=100)
ax = fig.add_axes([0.05, 0.05, 0.45, 0.9])

nb = 50
ypad = int(50*0.25)
nr = len(isort_all)
imraster = np.zeros((nr*(nb+ypad), 1200))
nk = 0
nks = []
for k,i in enumerate(isort_all):
    #print((sa==i).sum())
    raster = rasters_summary[sa==i].astype('float32') * 10
    lata = latenciesS[sa==i].copy()
    lata[lata >= 0.25] = np.nan
    
    nki = raster.shape[0]
    isort_raster = neural_sort[nk : nk+nki]
    
    raster = raster[isort_raster]
    raster = cv2.resize(raster, (1200, nb), cv2.INTER_LINEAR)
    imraster[k*(nb + ypad): k*(nb + ypad) + nb] = raster
    nk += nki
vmax = 12
im = ax.imshow(imraster, extent=[-30, 90, imraster.shape[0], 0], vmin=0, vmax=vmax, cmap='gray_r', aspect='auto')

ax.plot([0, 0], [-ypad/2, nr*(nb+ypad)], color='k')
ax.plot([30, 30], [-ypad/2, nr*(nb+ypad)], color='k')
ax.set_xticks(np.arange(-30, 91, 30))
ax.set_xlabel('time (s)')
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.set_yticks(np.arange(nb//2, (nr+0.1)*(nb+ypad), nb+ypad))
ax.set_yticklabels(ylabels)
ax.spines['top'].set_visible(False)
ax.set_ylim([-ypad/2, nr*(nb+ypad) - ypad/2])
ax.invert_yaxis()

apos = ax.get_position()
cax = fig.add_axes([apos.x1+0.03, apos.y1-0.05, 0.008, 0.05])
plt.colorbar(im, cax=cax)
cax.set_yticks([0,10])
cax.set_yticklabels(['0', '10'], fontsize=8)
cax.set_ylabel('firing rate (Hz)', fontsize=8)
cax.yaxis.tick_left()  

plot % active

In [ ]:
fig=plt.figure(figsize=(3,3), dpi=300)
ax=fig.add_subplot(1,1,1)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.plot(np.stack((np.zeros(len(fracC)), np.ones(len(fracC))), axis=0), 
        np.stack((100 - fracC * 100, 100 - fracS * 100), axis=0), color='k', zorder=-1, lw=0.5)
ax.scatter(np.zeros(len(fracC)), 100 - fracC * 100, s=8, color=colors)
ax.scatter(np.ones(len(fracC)), 100 - fracS * 100, s=8, color=colors)
#, fracS * 100)
ax.set_ylabel('% neurons active')
ax.set_xticks([0, 1])
ax.set_yticks(np.arange(0, 101, 25))
ax.set_xticklabels(['at random', 'at syncope onset'])
ax.set_ylim([0,102])
ax.set_xlim([-0.25, 1.25])
from scipy.stats import ttest_rel
ind = ~np.isnan(fracS)
ttest_rel(fracC[ind], fracS[ind])

## behavioral prediction analysis

compute residuals

In [ ]:
out = process.get_residuals(root)
laser_res, lasoff_res, pre_res, syncon_res, syncoff_res, rasters_res, rasters_S = out

plot residual summary

In [ ]:
from scipy.stats import ttest_rel

fig = plt.figure(figsize=(6,6), dpi=300)
ax = fig.add_subplot(1,1,1)
dy = [0, -18, -29]
txt = ['pre-laser', 'laser onset', 'syncope onset']
stars = ['***', ' ** ', ' * ', '']
pthr = [1e-3, 0.01, 0.05, 100]
ylabels = []
syncon_all=[]
for ip, vals in enumerate([pre_res, laser_res, syncon_res]):
    resk = np.zeros(len(isort_all))
    bottom = dy[ip]
    for k, i in enumerate(isort_all):
        ind = regions==i
        nn = ind.sum()
        res = np.mean(vals[ind]) * 10 #/ firing_rates[regions==i].mean()
        sres = (vals[ind] * 10).std() / (nn)**0.5 
        ax.errorbar(k, res + bottom, sres, color=colors[i])
        ax.bar(k, res, color=colors[i], bottom=bottom) 
        if ip > 0:
            p = ttest_rel(vals[ind], pre_res[ind]).pvalue
            istar = np.nonzero(p < pthr)[0][0]
            ax.text(k-0.5, res+np.sign(res)*(sres+1)+bottom, 
                    stars[istar], va='center', fontsize=10)
        #print(p < 0.05)
        #ax.set_ylim([-4, 15])
        if ip==0:
            ylabels.append(region_names[i])
        resk[k] = res + sres
        if ip==2:
            syncon_all.extend(vals[ind])
    k+=1
    ind = np.ones(len(regions), 'bool')
    res = np.mean(vals[ind]) * 10 #/ firing_rates[regions==i].mean()
    sres = (vals[ind] * 10).std() / (nn)**0.5 
    ax.errorbar(k, res + bottom, sres, color=[0.5,0.5,0.5])
    ax.bar(k, res, color=[0.5,0.5,0.5], bottom=bottom) 
    if ip > 0:
        p = ttest_rel(vals[ind], pre_res[ind]).pvalue
        istar = np.nonzero(p < pthr)[0][0]
        ax.text(k-0.5, res+np.sign(res)*(sres+1)+bottom, 
                stars[istar], va='center', fontsize=10)
    ax.plot([-1.4, -1.4], np.array([0, 2]) + bottom, color='k', lw=2)
    ax.plot([-1, k+1], np.ones(2) * bottom, color='k', lw=1)
    ax.text(-1, resk.max() + 1*(ip+1) + bottom, txt[ip], fontsize=14)
    
ax.set_xlim([-1.5, k+1])
ax.axis('off')
ax.text(-1.4, 0.2, '2 Hz', rotation=90, ha='right')#, transform=ax.transAxes)
for k,y in enumerate(ylabels):
    ax.text(k, bottom - 6, y, rotation=90, ha='center', va='top')
ax.text(k+1, bottom - 6, 'all', rotation=90, ha='center', va='top')
    

ax.text(-4, dy[-1]+ 2, 'zscore difference',  fontsize=14, rotation=90)
